# Oscillating Van der Pol Systems

In [1]:
import pysindy as ps
import numpy as np
import matplotlib.pyplot as plt
import dill
import sys
import os

from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import Lasso

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # include parent directory in the path
from data import SINDy_data
from data import data
from data import equations

sys.path.append("/home/mattg/D_CODE") # A questo punto è necessario per non doverlo chiamare i file che usano D_CODE.
from D_CODE.run_simulation import run as run_SRT
from D_CODE.run_simulation_vi import run as run_DCODE

from toolbox.auxiliary_functions import SRT_simulation, D_CODE_simulation, set_param_freq, intercept_library_fun, bb_combinations

# Seed:
np.random.seed(999)

/home/mattg/miniforge3/envs/SimbSINDy_env/lib/python3.11/site-packages/pysindy/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound


#### Case $\mu$, $A$ and $\omega$ fixed
Consider the modified Van der Pol system with a nonlinear forcing term
$$
\begin{cases}
\dot{x}_0 = \mu x_0 - x_1 - \mu x_0 x_1^2 + A\sin(\omega x_1^2) \\
\dot{x}_1 = x_0
\end{cases}
$$
and with $\mu=1$, $A=1$ and $\omega=1$ fixed.

In [4]:
# Select ODE & settings:

ode_name = 'OscilVdpODE' # help="name of the ode", type=str
ode_param = None # '1.,1.,1.' # help="parameters of the ode (default: None)", type=str, default=None
freq = 10 # help="sampling frequency", type=float, default=10
n_sample = 50 # help="number of trajectories", type=int, default=100
noise_sigma = 0.01 # help="noise level (default 0)", type=float, default=0.
seed = 100 # help="random seed", type=int, default=0
n_seed = 1 # help="random seed", type=int, default=10
alg = 'tv'

ode_param, freq = set_param_freq(ode_param, freq)

ode = equations.get_ode(ode_name, ode_param)
dt = 1 / freq
dim_x = 2
dim_k = 0
time_vector = np.arange(0, ode.T + dt, dt)
T = ode.T

In [ ]:
# # generate data:
# X_list, dX_list, param_list, feature_names = SINDy_data.SINDy_data(ode_name, ode_param, freq, n_sample, noise_sigma, dim_x, dim_k)
# print(np.shape(X_list), np.shape(dX_list), np.shape(param_list))
# print(feature_names)

# SINDy_data.plot_configuration(X_list, T)

In [ ]:
# additional building blocks -> running SR-T:
# building_blocks_lambda0, function_names0 = SRT_simulation(ode_name, ode_param, 0, freq, n_sample, noise_sigma, alg='tv', seed=seed, n_seed=n_seed, T=T)
# building_blocks_lambda1, function_names1 = SRT_simulation(ode_name, ode_param, 1, freq, n_sample, noise_sigma, alg='tv', seed=seed, n_seed=n_seed, T=T)

building_blocks_lambda0, function_names0 = D_CODE_simulation(ode_name, ode_param, 0, freq, n_sample, noise_sigma, seed=seed, n_seed=n_seed, T=T)
# building_blocks_lambda1, function_names1 = D_CODE_simulation(ode_name, ode_param, 1, freq, n_sample, noise_sigma, seed=seed, n_seed=n_seed, T=T)

NOTE: Building block sin(X1**2) trovato (!)

NOTE: Due equazioni: combinare i building blocks (anche se uno è nullo)

NOTE: Bisogna filtrare i building blocks perche i numeri negativi elevati alla potenza frazionaria sono numeri complessi -> NaN -> error!

NOTE: SINDy degree 3 in questo caso perche nel reale sistema ce un termine cubico: unico modo per trovare il modello corretto

In [ ]:
# combine the two blocks:
bbs, fns = bb_combinations(building_blocks_lambda0, building_blocks_lambda1, function_names0, function_names1, ode.init_high, ode.init_low, dim_x, dim_k)

valid_id = [1,3,4,6]

intercept_library = intercept_library_fun(dim_x+dim_k)
# polynomial library:
polynomial_library = ps.PolynomialLibrary(degree=3, include_bias=False)

errors = []
n_features_vec = []
for i in valid_id: # for i in range(len(bbs)):

    # building block library:
    custom_library = ps.CustomLibrary(library_functions=bbs[i], function_names=fns[i])

    # enlarged library, adding the building block to polynomial library:
    generalized_library = ps.GeneralizedLibrary(
        libraries=[polynomial_library, custom_library],
        tensor_array=[[1, 1]] 
    )

    # add the intercept:
    final_library = ps.ConcatLibrary([intercept_library, generalized_library])

    # fitting the model:
    model = ps.SINDy(feature_names=feature_names, feature_library=final_library, optimizer=ps.STLSQ(threshold=0.1))
    model.fit(X_list, t=dt, multiple_trajectories=True, x_dot=dX_list)

    # library:
    # print('')
    # print('library:')
    # library_terms = final_library.get_feature_names(input_features=feature_names)
    # for term in library_terms:
    #     print(term)

    # final model:
    print('')
    print('model:')
    model.print()

    # evaluate the model:
    
    # filter too complex models (for sure not correct and likely to crash the code):
    coefficients = model.coefficients()
    lasso_penalty = np.sum(np.abs(coefficients))
    if np.count_nonzero(np.array(model.coefficients())) < 10 and lasso_penalty < 10:

        # compute MSE:
        _, mse = SINDy_data.evaluate_RMSE(model, ode, freq, 10, ode.init_high, ode.init_low, dim_k)

        # lasso penalty:
        alpha = 0.01 # regularization parameter
        coefficients = model.coefficients()
        lasso_penalty = np.sum(np.abs(coefficients))

        # final evaluation metric:
        error = mse + alpha * lasso_penalty
        print('')
        print('error:', error)
    else:
        error = 1000
        print('')
        print('Too complex model')
    
    errors.append(error)
    n_features_vec.append(np.count_nonzero(np.array(model.coefficients())))
    print('')
    

print('errors:', errors)

In [ ]:
# Final model:
min_error = min(errors)
idxs = [i for i, e in enumerate(errors) if abs(e - min_error) < 0.01]
n_features_vec_2 = [n_features_vec[i] for i in idxs]

if len(idxs) > 1:
    print('Multiple models with similar error, choosing the simplest one')
    print('')
    # idx = idxs[np.argmin(n_features_vec_2)]
    idx = valid_id[idxs[np.argmin(n_features_vec_2)]]
else:
    # idx = idxs[0]
    idx = valid_id[idxs[0]]


# intercept library:
intercept_library = intercept_library_fun(dim_x+dim_k)
# polynomial library:
polynomial_library = ps.PolynomialLibrary(degree=3, include_bias=False)

# custom library with building blocks:
custom_library = ps.CustomLibrary(library_functions=bbs[idx], function_names=fns[idx])
model = ps.SINDy(feature_names=feature_names, feature_library=custom_library, optimizer=ps.STLSQ(threshold=0.01))
model.fit(X_list, t=dt, multiple_trajectories=True, x_dot=dX_list)
building_block = custom_library.get_feature_names(input_features=feature_names)

# enlarged library, adding building blocks to polynomial library:
generalized_library = ps.GeneralizedLibrary(
    libraries=[polynomial_library, custom_library],
    tensor_array=[[1, 1]] 
)

# add the intercept:
final_library = ps.ConcatLibrary([intercept_library, generalized_library])

# fitting the model:
model = ps.SINDy(feature_names=feature_names, feature_library=final_library, optimizer=ps.STLSQ(threshold=0.1))
model.fit(X_list, t=dt, multiple_trajectories=True, x_dot=dX_list)

# library:
print('')
length = 0
#print('library:')
library_terms = final_library.get_feature_names(input_features=feature_names)
for term in library_terms:
    print(term)
    length += 1
print('length:', length)
print('')


# best builging block:
print('Best building block:')
print(building_block)
print('')

# final model:
print('Symbolic-SINDy model:')
model.print()

In [ ]:
# evaluation
n_sample = 25
rmse_sigma, _ = SINDy_data.evaluate_RMSE(model, ode, freq, n_sample, ode.init_high, ode.init_low, dim_k)
print('Best Model RMSE: ', rmse_sigma)

n_sample = 1
title = r'No parameters'
SINDy_data.evaluate_traj(model, ode, freq, n_sample, [0.2, 0.8], [0.2, 0.8], dim_x, dim_k, title=title, T_aux=12)

In questo caso il sitema di ODE è stato identificato alla perfezione, giustificando l'impiego di una regressione simbolica per trovare potenziali building blocks rilevanti da mettere in SINDy.

Nota pero che cambiando i parametri, usando quindi parametri diversi da quelli di default, questo approcio fa lo stesso fatica, perché i metodi di Symbolic regression non riescono a beccare la vera dinamica, specialmente cambiando $\omega$ argomento del seno.

#### Case $\mu$, $A$ fixed and $\omega$ varying
Consider the modified Van der Pol system with a nonlinear forcing term
$$
\begin{cases}
\dot{x}_0 = \mu x_0 - x_1 - \mu x_0 x_1^2 + A\sin(\omega x_1^2) \\
\dot{x}_1 = x_0
\end{cases}
$$
and with $\mu=1$, $A=1$ fixed, and $\omega \in [1,\pi]$.

In [1]:
import pysindy as ps
import numpy as np
import matplotlib.pyplot as plt
import dill
import sys
import os

from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import Lasso

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # include parent directory in the path
from data import SINDy_data
from data import data
from data import equations

sys.path.append("/home/mattg/D_CODE") # A questo punto è necessario per non doverlo chiamare i file che usano D_CODE.
from D_CODE.run_simulation import run as run_SRT
from D_CODE.run_simulation_vi import run as run_DCODE

from toolbox.auxiliary_functions import SRT_simulation, D_CODE_simulation, set_param_freq, intercept_library_fun, bb_combinations

# Seed:
np.random.seed(999)

/home/mattg/miniforge3/envs/SimbSINDy_env/lib/python3.11/site-packages/pysindy/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound


Far variare $\omega$ è l'unica cosa sensata: gli altri termini concettualmente sono gia dentro i blocchi di SINDy perché semplice moltiplicazione tra la parametrizzazione e i block trovati, quindi non dovremmo avere difficoltà con SINDy, una volta identificato il $\sin(x^2)$. Il caso piu challenging è trovare la parametrizzazione sulla pulsazione.

In [2]:
# Select ODE & settings:
ode_name = 'OscilVdpODE_par_w' # help="name of the ode", type=str
ode_param = None # help="parameters of the ode (default: None)", type=str, default=None
x_id = 0 # help="ID of the equation to be learned", type=int, default=0
freq = 10 # help="sampling frequency", type=float, default=10
n_sample = 50 # help="number of trajectories", type=int, default=100
noise_sigma = 0.01 # help="noise level (default 0)", type=float, default=0.
seed = 100 # help="random seed", type=int, default=0
n_seed = 1 # help="random seed", type=int, default=10
alg = 'tv'

ode_param, freq = set_param_freq(ode_param, freq)

ode = equations.get_ode(ode_name, ode_param)
dt = 1 / freq
dim_x = 2
dim_k = 1
time_vector = np.arange(0, ode.T + dt, dt)
T = ode.T

In [ ]:
# generate data:
X_list, dX_list, param_list, feature_names = SINDy_data.SINDy_data(ode_name, ode_param, freq, n_sample, noise_sigma, dim_x, dim_k)
print(np.shape(X_list), np.shape(dX_list), np.shape(param_list))
print(feature_names)

SINDy_data.plot_configuration(X_list, T)

In [3]:
# additional building blocks -> running SR-T:
building_blocks_lambda0, function_names0 = SRT_simulation(ode_name, ode_param, 0, freq, n_sample, noise_sigma, alg='tv', seed=seed, n_seed=n_seed, T=T)
# building_blocks_lambda1, function_names1 = SRT_simulation(ode_name, ode_param, 1, freq, n_sample, noise_sigma, alg='tv', seed=seed, n_seed=n_seed, T=T)

Running with: ode_name=OscilVdpODE_par_w, ode_param=None, x_id=0, freq=10, n_sample=50, noise_sigma=0.01, alg=tv, seed=100, n_seed=1
Dataset shape:  (101, 50, 3)
Functions set:  {'neg': 1, 'mul': 1, 'log': 1, 'add': 1, 'sub': 1, 'pow': 1, 'sin': 1}
 
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     8.42      2.28657e+14        5         0.887382         0.842416     49.35s


/home/mattg/gplearn/gplearn/functions.py:132: RuntimeWarning: overflow encountered in power
  return np.where(np.logical_and(np.minimum(x1, x2) >= 0, np.maximum(x1, x2) <= 10), np.power(x1, x2), 1000)
/home/mattg/gplearn/gplearn/functions.py:132: RuntimeWarning: overflow encountered in power
  return np.where(np.logical_and(np.minimum(x1, x2) >= 0, np.maximum(x1, x2) <= 10), np.power(x1, x2), 1000)


   1     5.33      6.62137e+11       18         0.771179         0.774729     42.24s
   2     3.60          17095.5       18         0.771312         0.773533     32.07s
   3     5.23          21298.3       18         0.766542         0.816462     19.24s
   4     5.68           407073       18         0.764946         0.830819     13.53s
   5     6.40          233.409       11         0.727453         0.737242      0.00s

SR-T model:  -X1 + sin(sin(X1**2*X2))

Blocchi ultima generazione: [-X1 + sin(sin(X1**2*X2)), -X1, sin(sin(X1**2*X2)), sin(X1**2*X2), X1**2*X2, X1**2]
building blocks:
sin(X0 + X1)
sin(sin(X0 + X1))
sin(sin(X0))
sin(X1**4.271147555945891)
X1**4.271147555945891
-sin(X1 - sin(X0))
-X1 + sin(sin(X1**2*X2))
-X1
sin(sin(X1**2*X2))
sin(X1**2*X2)
X1**2*X2
X1**2
 
number of building blocks:  12
 


In [6]:
# additional building blocks -> running D-CODE:
building_blocks_lambda0, function_names0 = D_CODE_simulation(ode_name, ode_param, 0, freq, n_sample, noise_sigma, seed=seed, n_seed=n_seed, T=T)
# building_blocks_lambda1, function_names1 = D_CODE_simulation(ode_name, ode_param, 1, freq, n_sample, noise_sigma, seed=seed, n_seed=n_seed, T=T)

Running with: ode_name=OscilVdpODE_par_w, ode_param=None, x_id=0, freq=10, n_sample=50, noise_sigma=0.01, seed=100, n_seed=1
Dataset shape:  (201, 50, 3)
Functions set:  {'neg': 1, 'mul': 1, 'add': 1, 'sub': 1, 'pow': 1, 'sin': 1}
 
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     9.24      1.02145e+29        2          204.696          246.311      2.96m
   1     6.94      1.82847e+19        6          165.112          176.644      3.15m
   2     2.90      8.10006e+12        6          164.621          181.061      2.12m
 
promising programs:
sub(sin(add(X0, X1)), X1)
sub(sin(add(X0, X1)), X1)
add(sin(add(X0, X1)), neg(X1))
sub(sin(add(X0, X1)), X1)
sub(sin(add(X0, X1)), X1)
sub(sin(add(X0, X1)), X1)
add(sin(add(X0, X1)), neg(X1))
sub(sin(add(X0, X1)), X1)
sub(sin(add(X0, X1)), X1)
m

Commento finale:

La situazione è ben gestita nel primo caso, dove per parametri fissati (con quelli di default) allora in qualche modo il $\sin(x^2)$ salta fuori e aggiungendolo nella libreria di SINDy il modello trovato è quello vero, anche con rumore (es. 5%).

Tuttavia, cambiando i parametri da quelli di default, in particolare $\omega$ quel seno non viene piu beccato, e quindi la vera dinamica non viene mai ricostruita.
Per lo stesso motivo, analizzando la situazione per cui $\omega$ parametrizza la dinamica quel seno non viene visto. Probabilmente la dinamica è complessa vista anche la presenza di diverse combinazioni di X0 e X1, ed entrambi SR-T e D-CODE trovano dei loro building blocks o combinazioni di seni e coseni (che pero non dipendono da X2) oppure improbabili composizioni di seni, coseni e logaritmi. Come detto prima, la dinamica è complessa e la nonlinearità portata da diversi fattori, alla fine viene approssimata con un altro tipo di combinazioni in altre forme, specialmente da D-CODE.

NOTA: In questo caso abbiamo dovuto mettere a SINDy una libreria standard di grado 3, in modo da identificare correttamente la prima parte dell'espressione. Non sempre quel termine viene trovato dalla Symbolic Regression, ma spesso è incorporato insieme a un coseno o altre variabili.

NOTA sul codice gia esistente: Estraiamo solo un building block. Il che è sensato perché non ci aspettiamo che il modello reale sia eccessivamente complesso. In molti casi (questo o Selkov), alzando il degree di SINDy è sufficiente per recuperare la dinamica -> Considera però eventualmente il fatto che a volte potrebbe essere necessario estrarre più building blocks.